In [1]:
import pandas as pd
import janitor

In [2]:
# Group and keep columns with 'as_index'
# https://stackoverflow.com/questions/31569549/how-to-groupby-a-dataframe-in-pandas-and-keep-columns

### Daily report data

In [3]:
data = pd.read_csv('filter.csv')

In [4]:
print(data.set_index(['ID_REGISTRO']).index.is_unique) # ids are not unique... it's unclear how reports are aggregated

False


In [5]:
data['FECHA_INGRESO'] = pd.to_datetime(data.FECHA_INGRESO, format='%Y-%m-%d', errors='coerce')
data['FECHA_INGRESO'] = data['FECHA_INGRESO'].dt.strftime('%Y-%m-%d')
data = data.set_index(pd.DatetimeIndex(data['FECHA_INGRESO'])).sort_index()

In [6]:
len(data)

147221

### Note
~~Daily report for april 20 (ie, reported in 21 april) stops using testing site,~~
<br>
~~So we shall treat both sets of data accordingly~~

In [7]:
dat = data.copy()

In [8]:
tbl_x = dat.filter_date('FECHA_INGRESO','2020-01-06','2020-04-20')
tbl_x = tbl_x.drop(['ENTIDAD_RES'], axis=1) # drop by region
tbl_x = tbl_x.set_index(
    ['FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'ENTIDAD_UM', 'ID_REGISTRO', 'PAIS_ORIGEN']
)
tbl_x = tbl_x.groupby(
    ['FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'ENTIDAD_UM', 'ID_REGISTRO', 'PAIS_ORIGEN'], as_index=True).count().reset_index()
tbl_x = tbl_x.rename(columns={'ENTIDAD_UM':'Region_ID'})

In [9]:
tbl_y = dat.filter_date('FECHA_INGRESO','2020-04-20','2020-04-27')
tbl_y = tbl_y.drop(['ENTIDAD_UM'], axis=1) # drop by region
tbl_y = tbl_y.set_index(
    ['FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'ENTIDAD_RES', 'ID_REGISTRO', 'PAIS_ORIGEN']
)
tbl_y = tbl_y.groupby(
    ['FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'ENTIDAD_RES', 'ID_REGISTRO', 'PAIS_ORIGEN'], as_index=True).count().reset_index()
tbl_y = tbl_y.rename(columns={'ENTIDAD_RES':'Region_ID'})

In [10]:
tbl = pd.concat([tbl_x, tbl_y])

In [11]:
tbl

,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,Region_ID,ID_REGISTRO,PAIS_ORIGEN,RESULTADO
0,2020-01-06,2020-01-06,9999-99-99,18,09f6d9,Local,8
1,2020-01-08,2020-01-08,9999-99-99,30,1c29f8,Local,7
2,2020-01-27,2020-01-27,9999-99-99,15,1879e0,Local,1
3,2020-02-07,2020-02-07,9999-99-99,11,03e780,Local,5
4,2020-02-12,2020-02-07,9999-99-99,2,07b5c5,Local,8
...,...,...,...,...,...,...,...
4039,2020-04-26,2020-04-26,9999-99-99,28,1b5221,Local,1
4040,2020-04-26,2020-04-26,9999-99-99,28,1c6f5f,Local,1
4041,2020-04-26,2020-04-26,9999-99-99,28,1d28f9,Local,1
4042,2020-04-27,2020-04-20,9999-99-99,9,06ed10,Local,1


In [12]:
tbl = tbl.rename(
    columns={
        'FECHA_INGRESO':'Date_Confirmed',
        'FECHA_SINTOMAS':'Date_Symptoms',
        'ID_REGISTRO':'Case_ID',
        'RESULTADO':'Status',
        'PAIS_ORIGEN':'Origin'
    })

In [13]:
tbl['Status'] = '1'

In [14]:
tbl

,Date_Confirmed,Date_Symptoms,FECHA_DEF,Region_ID,Case_ID,Origin,Status
0,2020-01-06,2020-01-06,9999-99-99,18,09f6d9,Local,1
1,2020-01-08,2020-01-08,9999-99-99,30,1c29f8,Local,1
2,2020-01-27,2020-01-27,9999-99-99,15,1879e0,Local,1
3,2020-02-07,2020-02-07,9999-99-99,11,03e780,Local,1
4,2020-02-12,2020-02-07,9999-99-99,2,07b5c5,Local,1
...,...,...,...,...,...,...,...
4039,2020-04-26,2020-04-26,9999-99-99,28,1b5221,Local,1
4040,2020-04-26,2020-04-26,9999-99-99,28,1c6f5f,Local,1
4041,2020-04-26,2020-04-26,9999-99-99,28,1d28f9,Local,1
4042,2020-04-27,2020-04-20,9999-99-99,9,06ed10,Local,1


In [15]:
#tbl[~tbl.FECHA_DEF.str.startswith(('9999'))]
tbl = tbl[tbl.FECHA_DEF.str.startswith(('9999'))] # Not deceased

In [16]:
tbl = tbl.drop(['FECHA_DEF'], axis=1)

In [17]:
tbl

,Date_Confirmed,Date_Symptoms,Region_ID,Case_ID,Origin,Status
0,2020-01-06,2020-01-06,18,09f6d9,Local,1
1,2020-01-08,2020-01-08,30,1c29f8,Local,1
2,2020-01-27,2020-01-27,15,1879e0,Local,1
3,2020-02-07,2020-02-07,11,03e780,Local,1
4,2020-02-12,2020-02-07,2,07b5c5,Local,1
...,...,...,...,...,...,...
4039,2020-04-26,2020-04-26,28,1b5221,Local,1
4040,2020-04-26,2020-04-26,28,1c6f5f,Local,1
4041,2020-04-26,2020-04-26,28,1d28f9,Local,1
4042,2020-04-27,2020-04-20,9,06ed10,Local,1


In [18]:
#tbl.to_csv('confirmed_by_date.csv', index=False)

In [19]:
geo = pd.read_csv('../../data/dge/meta_data/entidades.csv')

In [20]:
geo.keys()

Index(['CLAVE_ENTIDAD', 'ENTIDAD_FEDERATIVA', 'ABREVIATURA'], dtype='object')

In [21]:
geo = geo.rename(columns={'CLAVE_ENTIDAD':'Region_ID', 'ENTIDAD_FEDERATIVA':'Region', 'ABREVIATURA':'Region_Key'})

In [28]:
output = pd.merge(tbl, geo, on='Region_ID') 

In [29]:
output = output[['Date_Confirmed', 'Date_Symptoms', 'Region', 'Region_ID', 'Region_Key', 'Origin']]

In [30]:
output.to_csv('../../latest.csv', index=False)

In [32]:
output.Origin.unique()

array(['Local', 'Estados Unidos de AmÃ©rica', 'Estados Unidos de América',
       'Venezuela', 'PerÃº', 'Perú', 'El Salvador', 'Cuba', 'Guatemala',
       'Canadá', 'Alemania', 'RepÃºblica de Honduras',
       'República de Honduras', 'Otro', 'CamerÃºn', 'Camerún', 'Chile',
       'Brasil'], dtype=object)